In [1]:
from layers import *
from units_struct import *
import tensorflow as tf

# W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))


is_training=True
batchnorm_enabled=False
l2_strength=0.01
num_groups=3
stage = 2
bias = 1.0
output_channels = {'1': [144, 288, 576], '2': [200, 400, 800], '3': [240, 480, 960], '4': [272, 544, 1088],
                                '8': [384, 768, 1536], 'conv1': 24}

## [batch_size, width, height, channel]
x = tf.Variable(tf.random_normal([1, 224, 224, 32], stddev=0.01))
residual = x
# bottleneck_filters = (output_channels[str(num_groups)][stage - 2] - residual.get_shape()[3].value) // 4


stage_layer = shufflenet_unit('stage' + str(stage) + '_0', x=x, w=None,
                                          num_groups=num_groups,
                                          group_conv_bottleneck=not (stage == 2),
                                          num_filters=
                                          output_channels[str(num_groups)][
                                              stage - 2],
                                          stride=(2, 2),
                                          fusion='concat', l2_strength=l2_strength,
                                          bias=bias,
                                          batchnorm_enabled=False,
                                          is_training=True)

residual_pooled = avg_pool_2d(residual, size=(3, 3), stride=(2,2), padding='SAME')

bottleneck = grouped_conv2d('Gbottleneck', x=x, w=None, num_filters=52, kernel_size=(1, 1),
                                        padding='VALID',
                                        num_groups=num_groups, l2_strength=l2_strength, bias=bias,
                                        activation=None,
                                        batchnorm_enabled=False, is_training=True)
shuffled = channel_shuffle('channel_shuffle', bottleneck, num_groups)



padded = tf.pad(shuffled, [[0, 0], [1, 1], [1, 1], [0, 0]], "CONSTANT")
depthwise = depthwise_conv2d('depthwise', x=padded, w=None, stride=(2,2), l2_strength=l2_strength,
                                     padding='VALID', bias=bias,
                                     activation=None, batchnorm_enabled=False, is_training=True)

group_conv1x1 = grouped_conv2d('Gconv1x1', x=depthwise, w=None,
                                           num_filters=240 - residual.get_shape()[3].value,
                                           kernel_size=(1, 1),
                                           padding='VALID',
                                           num_groups=num_groups, l2_strength=l2_strength, bias=bias,
                                           activation=None,
                                           batchnorm_enabled=False, is_training=True)
final = tf.concat([residual_pooled, group_conv1x1], axis=-1)
with tf.Session()as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    print(sess.run(tf.shape(stage_layer)))
    print(sess.run(tf.shape(residual_pooled)))
    print(sess.run(tf.shape(bottleneck)))
    print(sess.run(tf.shape(shuffled)))
    print(sess.run(tf.shape(padded)))
    print(sess.run(tf.shape(depthwise)))
    print(sess.run(tf.shape(group_conv1x1)))
    print(sess.run(tf.shape(final)))
    




    

    

[  1 112 112 239]
[  1 112 112  32]
[  1 224 224  51]
[  1 224 224  51]
[  1 226 226  51]
[  1 112 112  51]
[  1 112 112 207]
[  1 112 112 239]
